<h3>Generation of SALCs in Octahedral and Tetrahedral Coordination Environments</h3>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

<h5>Local Imports</h5>

In [2]:
from SALC_functions import *

Octahedral Symmetry Group, $O_h$

The operations which compose the octahedral point group, $O_h$, of an octahedron with vertices at integer positions nearest to the origin as follows

$$(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)$$

Will be stored in a dictionary for later use.

In [21]:
octahedral_vertices = np.array([[1,  0, -1,  0,  0,  0],
                                [0,  1,  0, -1,  0,  0],
                                [0,  0,  0,  0,  1, -1]], dtype=np.float64)

# A dictionary containing the 3x3 orthogonal matrices which represent the symmetry operations of the octahedral symmetry group
octahedron = {

#Identity
"E" : np.eye(3),

#Inversion
"i"  : -np.eye(3),

#60° roto-reflection about the triangular face normal
"S_1_6_1" : rotation([ 1, 1, 1], np.pi/3)@reflection([ 1, 1, 1]),
"S_1_6_2" : rotation([-1, 1, 1], np.pi/3)@reflection([-1, 1, 1]),
"S_1_6_3" : rotation([-1,-1, 1], np.pi/3)@reflection([-1,-1, 1]),
"S_1_6_4" : rotation([ 1,-1, 1], np.pi/3)@reflection([ 1,-1, 1]),

#300° roto-reflection about the triangular face normal
"S_5_6_1" : rotation([ 1, 1, 1], 5*np.pi/3)@reflection([ 1, 1, 1]),
"S_5_6_2" : rotation([-1, 1, 1], 5*np.pi/3)@reflection([-1, 1, 1]),
"S_5_6_3" : rotation([-1,-1, 1], 5*np.pi/3)@reflection([-1,-1, 1]),
"S_5_6_4" : rotation([ 1,-1, 1], 5*np.pi/3)@reflection([ 1,-1, 1]),

#120° rotation about the triangular face normal
"C_1_3_1" : rotation([ 1, 1, 1], 2*np.pi/3),
"C_1_3_2" : rotation([-1, 1, 1], 2*np.pi/3),
"C_1_3_3" : rotation([-1,-1, 1], 2*np.pi/3),
"C_1_3_4" : rotation([ 1,-1, 1], 2*np.pi/3),

#240° rotation about the triangular face normal
"C_2_3_1" : rotation([ 1, 1, 1], 4*np.pi/3),
"C_2_3_2" : rotation([-1, 1, 1], 4*np.pi/3),
"C_2_3_3" : rotation([-1,-1, 1], 4*np.pi/3),
"C_2_3_4" : rotation([ 1,-1, 1], 4*np.pi/3),

#90° roto-reflection about the principal axes
"S_1_4_1" : rotation([ 1, 0, 0], np.pi/2)@reflection([ 1, 0, 0]),
"S_1_4_2" : rotation([ 0, 1, 0], np.pi/2)@reflection([ 0, 1, 0]),
"S_1_4_3" : rotation([ 0, 0, 1], np.pi/2)@reflection([ 0, 0, 1]),

#270° roto-reflection about the principal axes
"S_3_4_1" : rotation([ 1, 0, 0], 3*np.pi/2)@reflection([ 1, 0, 0]),
"S_3_4_2" : rotation([ 0, 1, 0], 3*np.pi/2)@reflection([ 0, 1, 0]),
"S_3_4_3" : rotation([ 0, 0, 1], 3*np.pi/2)@reflection([ 0, 0, 1]),

#90° rotation about the principal axes
"C_1_4_1" : rotation([ 1, 0, 0], np.pi/2),
"C_1_4_2" : rotation([ 0, 1, 0], np.pi/2),
"C_1_4_3" : rotation([ 0, 0, 1], np.pi/2),

#270° rotation about the principal axes
"C_3_4_1" : rotation([ 1, 0, 0], 3*np.pi/2),
"C_3_4_2" : rotation([ 0, 1, 0], 3*np.pi/2),
"C_3_4_3" : rotation([ 0, 0, 1], 3*np.pi/2),

#180° rotation about the bisectors of the edges
"C_2_1" : rotation([ 1, 1, 0], np.pi),
"C_2_2" : rotation([-1, 1, 0], np.pi),
"C_2_3" : rotation([ 1, 0, 1], np.pi),
"C_2_4" : rotation([ 0, 1, 1], np.pi),
"C_2_5" : rotation([-1, 0, 1], np.pi),
"C_2_6" : rotation([ 0,-1, 1], np.pi),

#180° rotation about the pricipal axes
"C_2_x" : rotation([ 1, 0, 0], np.pi),
"C_2_y" : rotation([ 0, 1, 0], np.pi),
"C_2_z" : rotation([ 0, 0, 1], np.pi),

#mirror reflection across edges
"sigma_h_yz" : reflection([1,0,0]), 
"sigma_h_xz" : reflection([0,1,0]), 
"sigma_h_xy" : reflection([0,0,1]), 

#mirror reflection across edge midpoints
"sigma_d_1" : reflection([ 1, 1, 0]), 
"sigma_d_2" : reflection([-1, 1, 0]), 
"sigma_d_3" : reflection([ 1, 0, 1]), 
"sigma_d_4" : reflection([ 0, 1, 1]), 
"sigma_d_5" : reflection([-1, 0, 1]), 
"sigma_d_6" : reflection([ 0,-1, 1]), 

}

In [4]:
group_elements = list(octahedron.values())
n = len(group_elements)

#Check that none of the group elements are equal
for i in range(n):
    for j in range(i+1, n):
        if array_equal(group_elements[i], group_elements[j]):
            print("Elements {i}, {j} are equal".format(i=i, j=j))

#Check the closure of the group
for i in range(n):
    for j in range(n):
        product = group_elements[i] @ group_elements[j]
        included = False
        for k in range(n):
            if array_equal(product, group_elements[k]):
                included = True
                break
        if not included:
            print("The product of elements {i} and {j} is not in the group".format(i=i, j=j))

#Check that the vertices are invariant under the group actions
for i in range(n):
    transformed_vertices = group_elements[i] @ octahedral_vertices
    p = permutation_matrix(octahedral_vertices, transformed_vertices)
    if not valid_permutation_matrix(p):
        print("Element {i} does not leave the vertices invariant".format(i=i))

            

Tetrahedral Symmetry Group, $T_d$

The operations which compose the tetrahedral point group, $T_d$, of a tetrahedron with vertices at integer positions nearest to the origin as follows

$$(1, 0, -\frac{\sqrt{2}}{2}), (-1, 0, -\frac{\sqrt{2}}{2}), (0, 1, \frac{\sqrt{2}}{2}), (0, -1, \frac{\sqrt{2}}{2})$$

Will be stored in a dictionary for later use.

In [5]:
tetrahedral_vertices = np.array([[            1,           -1,            0,            0],
                                 [            0,            0,            1,           -1],
                                 [-np.sqrt(2)/2,-np.sqrt(2)/2, np.sqrt(2)/2, np.sqrt(2)/2]], dtype=np.float64)

# A dictionary containing the 3x3 orthogonal matrices which represent the symmetry operations of the octahedral symmetry group
tetrahedron = {

#Identity
"E" : np.eye(3),

#120° rotations about the vertices
"C_1_3_1" : rotation([ 1, 0, -np.sqrt(2)/2], 2*np.pi/3),
"C_1_3_2" : rotation([-1, 0, -np.sqrt(2)/2], 2*np.pi/3),
"C_1_3_3" : rotation([ 0, 1,  np.sqrt(2)/2], 2*np.pi/3),
"C_1_3_4" : rotation([ 0,-1,  np.sqrt(2)/2], 2*np.pi/3),

#240° rotations about the vertices
"C_2_3_1" : rotation([ 1, 0, -np.sqrt(2)/2], 4*np.pi/3),
"C_2_3_2" : rotation([-1, 0, -np.sqrt(2)/2], 4*np.pi/3),
"C_2_3_3" : rotation([ 0, 1,  np.sqrt(2)/2], 4*np.pi/3),
"C_2_3_4" : rotation([ 0,-1,  np.sqrt(2)/2], 4*np.pi/3),

#180° rotation about the edge bisectors
"C_1_2_1" : rotation([ 0, 0, 1], np.pi),
"C_1_2_2" : rotation([ 1, 1, 0], np.pi),
"C_1_2_3" : rotation([-1, 1, 0], np.pi),

#90° roto-reflection about the edge bisectors
"S_1_4_1" : rotation([ 0, 0, 1], np.pi/2)@reflection([ 0, 0, 1]),
"S_1_4_2" : rotation([ 1, 1, 0], np.pi/2)@reflection([ 1, 1, 0]),
"S_1_4_3" : rotation([-1, 1, 0], np.pi/2)@reflection([-1, 1, 0]),

#270° roto-reflection about the edge bisectors
"S_3_4_1" : rotation([ 0, 0, 1], 3*np.pi/2)@reflection([ 0, 0, 1]),
"S_3_4_2" : rotation([ 1, 1, 0], 3*np.pi/2)@reflection([ 1, 1, 0]),
"S_3_4_3" : rotation([-1, 1, 0], 3*np.pi/2)@reflection([-1, 1, 0]),

#mirror reflections
"sigma_d_1" : reflection([ 0, 1, 0]),
"sigma_d_2" : reflection([ np.sqrt(2),-np.sqrt(2), 2]),
"sigma_d_3" : reflection([ np.sqrt(2), np.sqrt(2), 2]),
"sigma_d_4" : reflection([ np.sqrt(2), np.sqrt(2),-2]),
"sigma_d_5" : reflection([-np.sqrt(2), np.sqrt(2), 2]),
"sigma_d_6" : reflection([ 1, 0, 0]),

}

In [6]:
group_elements = list(tetrahedron.values())
n = len(group_elements)

#Check that none of the group elements are equal
for i in range(n):
    for j in range(i+1, n):
        if array_equal(group_elements[i], group_elements[j]):
            print("Elements {i}, {j} are equal".format(i=i, j=j))

#Check the closure of the group
for i in range(n):
    for j in range(n):
        product = group_elements[i] @ group_elements[j]
        included = False
        for k in range(n):
            if array_equal(product, group_elements[k]):
                included = True
                break
        if not included:
            print("The product of elements {i} and {j} is not in the group".format(i=i, j=j))
            print(np.around(product, 3))

#Check that the vertices are invariant under the group actions
for i in range(n):
    transformed_vertices = group_elements[i] @ tetrahedral_vertices
    p = permutation_matrix(tetrahedral_vertices, transformed_vertices)
    if not valid_permutation_matrix(p):
        print("Element {i} does not leave the vertices invariant".format(i=i))

S-Orbital Representations

In [7]:
for key, element in octahedron.items():
    transformed_vertices = element @ octahedral_vertices
    p = permutation_matrix(octahedral_vertices, transformed_vertices)
    print(key, np.trace(p))

E 6
i 0
S_1_6_1 0
S_1_6_2 0
S_1_6_3 0
S_1_6_4 0
S_5_6_1 0
S_5_6_2 0
S_5_6_3 0
S_5_6_4 0
C_1_3_1 0
C_1_3_2 0
C_1_3_3 0
C_1_3_4 0
C_2_3_1 0
C_2_3_2 0
C_2_3_3 0
C_2_3_4 0
S_1_4_1 0
S_1_4_2 0
S_1_4_3 0
S_3_4_1 0
S_3_4_2 0
S_3_4_3 0
C_1_4_1 2
C_1_4_2 2
C_1_4_3 2
C_3_4_1 2
C_3_4_2 2
C_3_4_3 2
C_2_1 0
C_2_2 0
C_2_3 0
C_2_4 0
C_2_5 0
C_2_6 0
C_2_x 2
C_2_y 2
C_2_z 2
sigma_h_yz 4
sigma_h_xz 4
sigma_h_xy 4
sigma_d_1 2
sigma_d_2 2
sigma_d_3 2
sigma_d_4 2
sigma_d_5 2
sigma_d_6 2


In [8]:
for key, element in tetrahedron.items():
    transformed_vertices = element @ tetrahedral_vertices
    p = permutation_matrix(tetrahedral_vertices, transformed_vertices)
    print(key, np.trace(p))

E 4
C_1_3_1 1
C_1_3_2 1
C_1_3_3 1
C_1_3_4 1
C_2_3_1 1
C_2_3_2 1
C_2_3_3 1
C_2_3_4 1
C_1_2_1 0
C_1_2_2 0
C_1_2_3 0
S_1_4_1 0
S_1_4_2 0
S_1_4_3 0
S_3_4_1 0
S_3_4_2 0
S_3_4_3 0
sigma_d_1 2
sigma_d_2 2
sigma_d_3 2
sigma_d_4 2
sigma_d_5 2
sigma_d_6 2


In [15]:
elements = list(octahedron.values())
keys = list(octahedron.keys())
n = len(elements)

for key, element in octahedron.items():
    p = np.zeros((n,n), dtype=np.int8)
    for i in range(n):
        c = conjugate(element, elements[i])
        for j in range(n):
            if array_equal(c, elements[j]):
                p[i,j] = 1

    if not valid_permutation_matrix(p):
        print(key, " produces invalid permutation matrices")
    print(key, np.trace(p))
    key_list = []
    for i in range(n):
        if p[i,i]:
            key_list.append(keys[i])
    print(key_list)


    

E 48
['E', 'i', 'S_1_6_1', 'S_1_6_2', 'S_1_6_3', 'S_1_6_4', 'S_5_6_1', 'S_5_6_2', 'S_5_6_3', 'S_5_6_4', 'C_1_3_1', 'C_1_3_2', 'C_1_3_3', 'C_1_3_4', 'C_2_3_1', 'C_2_3_2', 'C_2_3_3', 'C_2_3_4', 'S_1_4_1', 'S_1_4_2', 'S_1_4_3', 'S_3_4_1', 'S_3_4_2', 'S_3_4_3', 'C_1_4_1', 'C_1_4_2', 'C_1_4_3', 'C_3_4_1', 'C_3_4_2', 'C_3_4_3', 'C_2_1', 'C_2_2', 'C_2_3', 'C_2_4', 'C_2_5', 'C_2_6', 'C_2_x', 'C_2_y', 'C_2_z', 'sigma_h_yz', 'sigma_h_xz', 'sigma_h_xy', 'sigma_d_1', 'sigma_d_2', 'sigma_d_3', 'sigma_d_4', 'sigma_d_5', 'sigma_d_6']
i 48
['E', 'i', 'S_1_6_1', 'S_1_6_2', 'S_1_6_3', 'S_1_6_4', 'S_5_6_1', 'S_5_6_2', 'S_5_6_3', 'S_5_6_4', 'C_1_3_1', 'C_1_3_2', 'C_1_3_3', 'C_1_3_4', 'C_2_3_1', 'C_2_3_2', 'C_2_3_3', 'C_2_3_4', 'S_1_4_1', 'S_1_4_2', 'S_1_4_3', 'S_3_4_1', 'S_3_4_2', 'S_3_4_3', 'C_1_4_1', 'C_1_4_2', 'C_1_4_3', 'C_3_4_1', 'C_3_4_2', 'C_3_4_3', 'C_2_1', 'C_2_2', 'C_2_3', 'C_2_4', 'C_2_5', 'C_2_6', 'C_2_x', 'C_2_y', 'C_2_z', 'sigma_h_yz', 'sigma_h_xz', 'sigma_h_xy', 'sigma_d_1', 'sigma_d_2', 's

Defining a Group class will make the following math much easier

In [129]:

class Group:

    def __init__(self, elements: list, eps=0.001):
        self.elements = elements
        self.eps = eps
        self._close()
        self._multiplication_table()

    def __len__(self):
        """return the order of the group"""
        return len(self.elements)
    
    def __contains__(self, other):
        for element in self.elements:
            if array_equal(other, element, eps=self.eps):
                return True
        return False
    
    def __getitem__(self, index):
        """return the corresponding element given an index or slice"""
        return self.elements[index]
    
    def __add__(self, other_group):
        """return the prodcut of two group"""
        return Group(self.elements + other_group.elements)
    
    def __eq__(self, other_group):
        """return True if two groups have the same elements"""
        for element in self.elements:
            if not element in other_group:
                return False
        return True

    def _close(self):
        """ensure group is closed under matrix multiplication"""
        closed = False
        while not closed:
            product_list = []
            n = len(self)
            for i in range(n):
                for j in range(n):
                    product = self[i] @ self[j]
                    if (not product in self) and (not np.array([array_equal(product, p) for p in product_list]).sum()):
                        product_list.append(product)
            if len(product_list) == 0:
                closed = True
            else:
                self.elements = self.elements + product_list

    def _multiplication_table(self):
        """store the multiplication table of the group in terms on element indices to use as a LU table"""
        n = len(self)
        self._mult_table = np.zeros((n,n), dtype=np.uint16)
        for i in range(n):
            for j in range(n):
                product = self.elements[i] @ self.elements[j]
                for k in range(n):
                    if array_equal(product, self.elements[k]):
                        self._mult_table[i,j] = k 
        
    def element_index(self, element: np.ndarray):
        """return the index associated with an element"""
        n = len(self)
        for i in range(n):
            if array_equal(element, elements[i]):
                return i
        raise Exception

    def create_subgroup(self, index_list: list):
        """return the subgroup corresponding to the given indices"""
        element_list = [self.elements[index] for index in index_list]
        return Group(element_list)
    

    def is_subgroup(self, group: Group):
        """return True if other group is a subgroup"""
        for element in group.elements:
            if not element in self:
                return False
        return True
    
    def is_self_conjugate(self, subgroup: Group):
        """check if the group is self conjugate"""
        for element in self:
            for sub_el in subgroup:
                conjugate = element @ sub_el @ np.linalg.inv(element)
                if not conjugate in subgroup:
                    return False
            return True

    def coset(self, subgroup: Group, index: int, side="right"):
        """return the coset indices given the subgroup indices and element index"""
        index_list = []
        if side == "left":
            for sub_element in subgroup.elements:
                product = self.elements[index] @ sub_element
                index_list.append(self.element_index(product))
        elif side == "right":
            for sub_element in subgroup.elements:
                product = sub_element @ self.elements[index] 
                index_list.append(self.element_index(product))    
        return index_list
    
    def unique_cosets(self, subgroup: Group, side="right"):
        """return the list of unique cosets given a subgroup"""
        unique_coset_list = []
        for i, element in enumerate(self.elements):
            coset_indices = sorted(self.coset(subgroup, i))
            if not coset_indices in unique_coset_list:
                unique_coset_list.append(coset_indices)
        return unique_coset_list

    def coset_multiply(self, cs1: list, cs2: list):
        """given two lists of coset indices return their product"""
        product_list = []
        for i in cs1:
            for j in cs2:
                product_index = self._mult_table[i,j] 
                if not product_index in product_list:
                    product_list.append(product_index)


    def conjugacy_class_indices(self, index: int):
        """find the conjugacy class given the index"""
        conjugate_list = []
        for element in self.elements:
            conjugate = element @ self.elements[index] @ np.linalg.inv(element)
            c_index = self.element_index(conjugate)
            if not c_index in conjugate_list:
                conjugate_list.append(c_index)
        return conjugate_list
            

    def conjugacy_class_list(self):
        """return the list of unique conjugacy classes"""
        class_list = []
        for i in range(len(self)):
            conjugacy_class = sorted(self.conjugacy_class_indices(i))
            if not conjugacy_class in class_list:
                class_list.append(conjugacy_class)
        return class_list
    



In [95]:
def regular_representation(index: int, group: Group) -> np.ndarray:
    """given a group, return the regular representation in the form of a binary matrix"""
    n = len(group)
    reg_rep = np.zeros((n,n), dtype=np.int8)
    for i in range(n):
        for j in range(n):
            product = group.elements[i] @ group.elements[j]
            reg_rep[i,j] = array_equal(product, group[index])
    return reg_rep

In [96]:
def point_representation(index: int, group: Group, vertices: np.ndarray) -> np.ndarray:
    """return a representation of the transformation of the vertices under the group actions"""
    _, basis_dim = vertices.shape
    transformed_vertices = group[index] @ vertices
    point_rep = np.zeros((basis_dim, basis_dim), dtype=np.int8)
    for i in range(basis_dim):
        for j in range(basis_dim):
            point_rep[i,j] = array_equal(vertices[:,i], transformed_vertices[:,j])
    return point_rep

In [97]:
oct = list(octahedron.values())
g = Group(oct)
ccl = g.conjugacy_class_list()

class_orders = []
trace_list = []
for cls in ccl:
    class_orders.append(len(cls))
    trace_list.append(np.trace(point_representation(cls[0], g, octahedral_vertices)))


print(class_orders)
print(trace_list)

[1, 1, 8, 8, 6, 6, 6, 3, 3, 6]
[6, 0, 0, 0, 0, 2, 0, 2, 4, 2]


In [99]:
ccl[2]

[2, 3, 4, 5, 6, 7, 8, 9]

In [103]:
for index in ccl[2]:
    print(np.around(g[index], 3),"\n")

[[-0. -1. -0.]
 [-0. -0. -1.]
 [-1. -0. -0.]] 

[[-0.  0.  1.]
 [ 1. -0. -0.]
 [ 0. -1. -0.]] 

[[-0. -1.  0.]
 [-0. -0.  1.]
 [ 1.  0. -0.]] 

[[-0.  0. -1.]
 [ 1. -0.  0.]
 [-0.  1. -0.]] 

[[-0. -0. -1.]
 [-1. -0. -0.]
 [-0. -1. -0.]] 

[[-0.  1.  0.]
 [ 0. -0. -1.]
 [ 1. -0. -0.]] 

[[-0. -0.  1.]
 [-1. -0.  0.]
 [ 0.  1. -0.]] 

[[-0.  1. -0.]
 [ 0. -0.  1.]
 [-1.  0. -0.]] 



In [133]:
for cls in ccl:
    elements = [g[index] for index in cls]
    sub = Group(elements)
    print(len(cls), len(sub), g.is_self_conjugate(sub))


1 1 True
1 2 True
8 24 True
8 12 True
6 24 True
6 24 True
6 24 True
3 4 True
3 8 True
6 24 True


In [132]:
oct = list(octahedron.values())
g = Group(oct)

subgroup = g.create_subgroup([2,3])
len(subgroup)

g.is_subgroup(subgroup)
print(len(subgroup))


24


In [109]:
for element in subgroup.elements:
    print(np.around(element, 3), "\n")

[[-0. -1. -0.]
 [-0. -0. -1.]
 [-1. -0. -0.]] 

[[-0.  0.  1.]
 [ 1. -0. -0.]
 [ 0. -1. -0.]] 

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]] 

[[-1.  0. -0.]
 [-0.  1. -0.]
 [ 0. -0. -1.]] 

[[-1.  0. -0.]
 [-0. -1.  0.]
 [ 0.  0.  1.]] 

[[ 0. -1. -0.]
 [-0.  0.  1.]
 [-1.  0.  0.]] 

[[-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]] 

[[ 0. -1.  0.]
 [ 0. -0.  1.]
 [ 1. -0.  0.]] 

[[ 0.  1. -0.]
 [-0.  0. -1.]
 [ 1.  0.  0.]] 

[[ 0. -0. -1.]
 [ 1.  0. -0.]
 [-0.  1.  0.]] 

[[ 0.  1. -0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]] 

[[ 0.  0. -1.]
 [-1.  0. -0.]
 [ 0. -1.  0.]] 

[[ 0. -0.  1.]
 [-1.  0. -0.]
 [-0.  1. -0.]] 

[[-1.  0.  0.]
 [ 0. -1. -0.]
 [ 0. -0. -1.]] 

[[-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]] 

[[ 0. -1.  0.]
 [ 0. -0.  1.]
 [ 1. -0.  0.]] 

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]] 

[[-0.  0. -1.]
 [-1.  0. -0.]
 [-0.  1. -0.]] 

[[-0. -0.  1.]
 [-1. -0.  0.]
 [-0. -1.  0.]] 

[[ 0.  1. -0.]
 [-0.  0. -1.]
 [ 1.  0.  0.]] 

[[ 0. -0. -1.]
 [ 1.  0. -0.]
 [-0.  1.  0.]] 

[[-0. -0. 

Find the Conjugacy Classes

Defining the SALC projection operator

S-orbitals in an octahedral coordination environment